In [146]:
import datetime
from datetime import timedelta
import re

### This takes like 40 minutes
- Could be interesting to rewrite in Rust and see how long that takes

In [192]:
fh = open('log.log', 'r')
log = re.compile(r'.* (.*) - (?:INFO|DEBUG) - (.*)')

time = 'a'
cur = 0
var = [timedelta() for _ in range(5)]
i = 0
loop_limit = 8306000
while (i < loop_limit):

    info = log.findall(fh.readline())
    i += 1
    if len(info) > 0:
        t, m = info[0]
    else:
        continue

    if any(m.startswith(x) for x in ['Executing', 'Creating xml', 'Creating images', 'Parsing the xml', 'Parsing the images']):

        t = datetime.datetime.strptime(t, '%H:%M:%S,%f')
        # Finish the previous section
        if not isinstance(time, str):
            var[cur] += (t - time)

        # Note the current session and start time
        if m.startswith('Executing'):
            cur = 4
            # # just loop here until a 'Found' is found
            while((not m.startswith('Found')) and (i < loop_limit)):
                info = log.findall(fh.readline())
                i += 1
                if len(info) > 0:
                    _, m = info[0]
                else:
                    continue
        elif m.startswith('Creating xml'):
            cur = 0
        elif m.startswith('Creating images'):
            cur = 1
        elif m.startswith('Parsing the xml'):
            cur = 2
        elif m.startswith('Parsing the images'):
            cur = 3
        
        time = t
    
    
print(f"XML Conversion: \t{var[0]}")
print(f"Image Conversion: \t{var[1]}")
print(f"XML Parsing: \t\t{var[2]}")
print(f"Image Parsing: \t\t{var[3]}")
print(f"SQL Time: \t\t{var[4]}")

XML Conversion: 	0:00:37.057000
Image Conversion: 	2:54:33.971000
XML Parsing: 		0:00:17.608000
Image Parsing: 		1:28:13.917000
SQL Time: 		0:07:34.025000


### Percentage of time

In [206]:
por = [y.total_seconds() * 100 / sum([x.total_seconds() for x in var]) for y in var]
por

[0.22767070572205045,
 64.34995734361362,
 0.10817998721844359,
 32.52475428188899,
 2.7894376815568975]